In [40]:
import pandas as pd
import numpy as np
import os 

# Set max columns
pd.set_option('display.max_columns', None)

DATADIR = 'data/2020'
OTHERDIR = 'data/misc/'

# Set paths
DATASET = os.path.join(DATADIR, '2020.csv')
DATAOUTPUT =  os.path.join(DATADIR, '2020pp.csv')
HIERARCHY = os.path.join(OTHERDIR, 'hierarchy.xlsx')

In [8]:
df = pd.read_csv(DATASET, index_col=[0])

/usr/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
df

,AUDI,DECL,DPTO,ELE,GVD,INC,INOFFDPTO,INOFFUSI,LOC,MADU,PJI,RETO,TALL,TIPO,UET,ZONA
0,CAPS,2020-01-27 09:00:00.000000,SOLD,4120,V2,180,NaN,NaN,NaN,NaN,655200530232,NaN,NaN,NaN,NaN,NaN
1,CAPS,2020-01-27 09:00:00.000000,PINT,4466,V2,1010,NaN,NaN,NaN,NaN,655200420197,NaN,NaN,NaN,NaN,NaN
2,CAPS,2020-01-27 09:00:00.000000,PINT,4420,V2,1110,NaN,NaN,NaN,NaN,655200450172,NaN,NaN,NaN,NaN,NaN
3,CAPS,2020-01-27 09:00:00.000000,PINT,4412,V2,1180,NaN,NaN,NaN,NaN,655200450172,NaN,NaN,NaN,NaN,NaN
4,CAPS,2020-01-27 09:00:00.000000,PINT,4414,V2,1270,NaN,NaN,NaN,NaN,655200420197,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235533,PESD,2020-12-30 09:06:00,PINT,4502,V2,1000,NaN,NaN,GRAL,NaN,655210140012,NaN,NaN,ASP,DIP,PESD
235534,PESD,2020-12-30 10:03:00,SOLD,7887,V2,3220,NaN,NaN,NaN,NaN,655210140033,NaN,NaN,GMTR,659,PESD
235535,PESD,2020-12-30 10:03:00,MONT,5215,V2,4450,NaN,NaN,K74,NaN,655210140033,NaN,NaN,NCON,654,PESD
235536,PESD,2020-12-30 10:03:00,PINT,4401,V2,1190,NaN,NaN,H54,NaN,655210140033,NaN,NaN,ASP,483,PESD


# Fill DPTO, TALL, UET
Debido a la estructura jerarquica que se nos presenta, iremos completando los datos desde la UET hacía el DPTO debido a que algunos datos no son correctos.

In [10]:
hie = pd.read_excel(HIERARCHY, 'Datos')
hie.drop(columns=['DPTO', 'TALL', 'UET'], inplace=True)
hie.rename(columns={'DPTOACRO':'DPTO','TALLACRO':'TALL','UETACRO':'UET'}, inplace=True)

In [11]:
hie['DPTO'] = hie['DPTO'].astype(str)
hie['TALL'] = hie['TALL'].astype(str)
hie['UET'] = hie['UET'].astype(str)

In [12]:
hie['DPTO'].replace({'DCHAP':'SOLD', 'DPINT':'PINT', 'EMB ':'EMBU', 'DMONT':'MONT', 'MONTVU ':'MONT', 'CALID':'CALI'}, inplace=True)

In [13]:
# Completar taller en función a la UET
def fill_taller(uet):
    hieuet = hie.copy()
    hieuet.set_index('UET', inplace=True)
    try:
        return hieuet.loc[uet].TALL
    except:
        pass
    
df['TALL'] = df.apply(lambda row: fill_taller(row['UET']), axis=1)

In [14]:
df

,AUDI,DECL,DPTO,ELE,GVD,INC,INOFFDPTO,INOFFUSI,LOC,MADU,PJI,RETO,TALL,TIPO,UET,ZONA
0,CAPS,2020-01-27 09:00:00.000000,SOLD,4120,V2,180,NaN,NaN,NaN,NaN,655200530232,NaN,None,NaN,NaN,NaN
1,CAPS,2020-01-27 09:00:00.000000,PINT,4466,V2,1010,NaN,NaN,NaN,NaN,655200420197,NaN,None,NaN,NaN,NaN
2,CAPS,2020-01-27 09:00:00.000000,PINT,4420,V2,1110,NaN,NaN,NaN,NaN,655200450172,NaN,None,NaN,NaN,NaN
3,CAPS,2020-01-27 09:00:00.000000,PINT,4412,V2,1180,NaN,NaN,NaN,NaN,655200450172,NaN,None,NaN,NaN,NaN
4,CAPS,2020-01-27 09:00:00.000000,PINT,4414,V2,1270,NaN,NaN,NaN,NaN,655200420197,NaN,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235533,PESD,2020-12-30 09:06:00,PINT,4502,V2,1000,NaN,NaN,GRAL,NaN,655210140012,NaN,SERVP,ASP,DIP,PESD
235534,PESD,2020-12-30 10:03:00,SOLD,7887,V2,3220,NaN,NaN,NaN,NaN,655210140033,NaN,TOLE4,GMTR,659,PESD
235535,PESD,2020-12-30 10:03:00,MONT,5215,V2,4450,NaN,NaN,K74,NaN,655210140033,NaN,CH-ME,NCON,654,PESD
235536,PESD,2020-12-30 10:03:00,PINT,4401,V2,1190,NaN,NaN,H54,NaN,655210140033,NaN,PTAL2,ASP,483,PESD


In [15]:
# Completar DPTO en función al taller
def fill_dpto(tall):
    hietall = hie.copy()
    hietall = hietall.drop(columns='UET').drop_duplicates(subset='TALL')
    hietall.set_index('TALL', inplace=True)
    try:
        return hietall.loc[tall].DPTO
    except:
        pass

df['DPTO'] = df.apply(lambda row: fill_dpto(row['TALL']), axis=1)

In [16]:
df

,AUDI,DECL,DPTO,ELE,GVD,INC,INOFFDPTO,INOFFUSI,LOC,MADU,PJI,RETO,TALL,TIPO,UET,ZONA
0,CAPS,2020-01-27 09:00:00.000000,None,4120,V2,180,NaN,NaN,NaN,NaN,655200530232,NaN,None,NaN,NaN,NaN
1,CAPS,2020-01-27 09:00:00.000000,None,4466,V2,1010,NaN,NaN,NaN,NaN,655200420197,NaN,None,NaN,NaN,NaN
2,CAPS,2020-01-27 09:00:00.000000,None,4420,V2,1110,NaN,NaN,NaN,NaN,655200450172,NaN,None,NaN,NaN,NaN
3,CAPS,2020-01-27 09:00:00.000000,None,4412,V2,1180,NaN,NaN,NaN,NaN,655200450172,NaN,None,NaN,NaN,NaN
4,CAPS,2020-01-27 09:00:00.000000,None,4414,V2,1270,NaN,NaN,NaN,NaN,655200420197,NaN,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235533,PESD,2020-12-30 09:06:00,PINT,4502,V2,1000,NaN,NaN,GRAL,NaN,655210140012,NaN,SERVP,ASP,DIP,PESD
235534,PESD,2020-12-30 10:03:00,SOLD,7887,V2,3220,NaN,NaN,NaN,NaN,655210140033,NaN,TOLE4,GMTR,659,PESD
235535,PESD,2020-12-30 10:03:00,MONTVU,5215,V2,4450,NaN,NaN,K74,NaN,655210140033,NaN,CH-ME,NCON,654,PESD
235536,PESD,2020-12-30 10:03:00,PINT,4401,V2,1190,NaN,NaN,H54,NaN,655210140033,NaN,PTAL2,ASP,483,PESD


In [22]:
# Set type for cols
#df['PJI'] = df['PJI'].astype(int).astype(str)
df['ELE'] = df['ELE'].astype(str).str.zfill(4)
df['INC'] = df['INC'].astype(str).str.zfill(4)
df['LOC'] = df['LOC'].astype(str)

In [27]:
df.DPTO.replace({'MONTVU':'MONT'}, inplace=True)

In [28]:
df.GVD.unique()

array(['V2', 'V1', 'V1E', '0', 'v2'], dtype=object)

In [39]:
# Data homogenization
df['GVD'].replace({'P2':'V2', 
                   '0':'V2',
                   'v2':'V2',
                   'V2F':'V2',
                   'V1E':'V1',
                   '0':'V2',
                   'v2':'V2',
                   'V1+':'V1',
                   'V1C':'V1',
                   'V3':'V2',
                   'P1':'V1',
                   'V1E':'V1',
                   'V1F':'V1',
                   'V1C':'V1',
                   'V1B':'V1',
                   'V1D':'V1',
                   'V1A':'V1',
                   'V2B':'V2'}, inplace=True)

In [33]:
df = df[df['DECL']>='2020-01-01']

In [36]:
df = df[df['DECL']<'2021-01-01']

In [37]:
max(df.DECL)

'2020-12-30 14:35:16.999996'

In [41]:
df.to_csv(DATAOUTPUT)